In [ ]:
!pip install tensorflow==2.9.2 transformers==4.18.0 datarobot==2.28.0
from transformers import AutoTokenizer, TFBertForQuestionAnswering
import numpy as np
from pprint import pprint

In [ ]:
FOUNDATION_MODEL = "bert-large-uncased-whole-word-masking-finetuned-squad"

# Get BERT for question answering

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(FOUNDATION_MODEL)
model = TFBertForQuestionAnswering.from_pretrained(FOUNDATION_MODEL)
BASE_PATH = "/content/blog_aix"
tokenizer.save_pretrained(BASE_PATH)
model.save_pretrained(BASE_PATH)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


# Deploy to DataRobot

## Assemble inference scripts & dependencies

In [ ]:
%%writefile $BASE_PATH/custom.py
"""
Copyright 2021 DataRobot, Inc. and its affiliates.
All rights reserved.
This is proprietary source code of DataRobot, Inc. and its affiliates.
Released under the terms of DataRobot Tool and Utility Agreement.
"""
import json
import os.path
import os
import tensorflow as tf
import pandas as pd
from transformers import AutoTokenizer, TFBertForQuestionAnswering
import io

def load_model(input_dir):
    tokenizer = AutoTokenizer.from_pretrained(input_dir)
    tf_model = TFBertForQuestionAnswering.from_pretrained(
        input_dir, return_dict=True
    )
    return tf_model, tokenizer


def log_for_drum(msg):
    os.write(1, f"\n{msg}\n".encode("UTF-8"))


def _get_answer_in_text(output, input_ids, idx, tokenizer):
    answer_start = tf.argmax(output.start_logits, axis=1).numpy()[idx]
    answer_end = (tf.argmax(output.end_logits, axis=1) + 1).numpy()[idx]
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
    )
    return answer


def score_unstructured(model, data, query, **kwargs):
    global model_load_duration
    tf_model, tokenizer = model

    # Assume batch input is sent with mimetype:"text/csv"
    # Treat as single prediction input if no mimetype is set
    is_batch = kwargs["mimetype"] == "text/csv"

    if is_batch:
        input_pd = pd.read_csv(io.StringIO(data), sep="|")
        input_pairs = list(zip(input_pd["abstract"], input_pd["question"]))

        start = time.time()
        inputs = tokenizer.batch_encode_plus(
            input_pairs, add_special_tokens=True, padding=True, return_tensors="tf"
        )
        input_ids = inputs["input_ids"].numpy()
        output = tf_model(inputs)
        responses = []
        for i, row in input_pd.iterrows():
            answer = _get_answer_in_text(output, input_ids[i], i, tokenizer)
            response = {
                "abstract": row["abstract"],
                "question": row["question"],
                "answer": answer,
            }
            responses.append(response)
        pred_duration = time.time() - start
        to_return = json.dumps(
            {
                "predictions": responses,
                "pred_duration": pred_duration,
            }
        )
    else:
        data_dict = json.loads(data)
        abstract, question = data_dict["abstract"], data_dict["question"]
        start = time.time()
        inputs = tokenizer(
            question,
            abstract,
            add_special_tokens=True,
            padding=True,
            return_tensors="tf",
        )
        input_ids = inputs["input_ids"].numpy()[0]
        output = tf_model(inputs)
        answer = _get_answer_in_text(output, input_ids, 0, tokenizer)
        pred_duration = time.time() - start
        to_return = json.dumps(
            {
                "abstract": abstract,
                "question": question,
                "answer": answer,
                "pred_duration": pred_duration,
            }
        )
    return to_return

Overwriting /content/blog_aix/custom.py


In [ ]:
%%writefile $BASE_PATH/requirements.txt
transformers

Overwriting /content/blog_aix/requirements.txt


## Upload using DataRobot Python API

In [ ]:
def deploy_to_datarobot(folder_path, env_name, model_name, descr):
  API_TOKEN = "YOUR_API_TOKEN"
  import datarobot as dr
  dr.Client(token=API_TOKEN, endpoint='https://app.datarobot.com/api/v2/')
  
  keras_execution_env = dr.ExecutionEnvironment.list(search_for=env_name)[0]
  custom_model = dr.CustomInferenceModel.create(
      name=model_name,
      target_type=dr.TARGET_TYPE.UNSTRUCTURED,
      description=descr,
      language='python'
  )
  print(f"Creating custom model version on {keras_execution_env}...")
  
  model_version = dr.CustomModelVersion.create_clean(
      custom_model_id=custom_model.id,
      base_environment_id=keras_execution_env.id,
      folder_path=folder_path,
      maximum_memory=4096 * 1024 * 1024,
  )
  print(f"Created {model_version}.")

  versions = dr.CustomModelVersion.list(custom_model.id)
  sorted_versions = sorted(versions, key=lambda v: v.label)
  latest_version = sorted_versions[-1]

  print("Building the execution environment with dependency packages...")
  build_info = dr.CustomModelVersionDependencyBuild.start_build(
      custom_model_id=custom_model.id,
      custom_model_version_id=latest_version.id,
      max_wait=3600,
  )
  print(f"Environment build completed with {build_info.build_status}.")

  print("Creating model deployment...")
  default_prediction_server = dr.PredictionServer.list()[0]
  deployment = dr.Deployment.create_from_custom_model_version(latest_version.id,
                                                              label=model_name,
                                                              description=descr,
                                                              default_prediction_server_id=default_prediction_server.id,
                                                              max_wait=600,
                                                              importance=None)
  
  print(f"{deployment} is ready!")
  return deployment

In [ ]:
"""
Usage:
    python datarobot-predict.py <input-file> [mimetype] [charset]

This example uses the requests library which you can install with:
    pip install requests
We highly recommend that you update SSL certificates with:
    pip install -U urllib3[secure] certifi
"""
import sys
import json
import requests

API_URL = 'https://mlops-dev.dynamic.orm.datarobot.com/predApi/v1.0/deployments/{deployment_id}/predictionsUnstructured'    # noqa
API_KEY = 'YOUR_API_TOKEN'
DATAROBOT_KEY = 'YOUR_DATAROBOT_KEY'

# Don't change this. It is enforced server-side too.
MAX_PREDICTION_FILE_SIZE_BYTES = 52428800  # 50 MB


class DataRobotPredictionError(Exception):
    """Raised if there are issues getting predictions from DataRobot"""


def make_datarobot_deployment_unstructured_predictions(data, deployment_id, mimetype, charset):
    """
    Make unstructured predictions on data provided using DataRobot deployment_id provided.
    See docs for details:
         https://app.datarobot.com/docs/predictions/api/dr-predapi.html

    Parameters
    ----------
    data : bytes
        Bytes data read from provided file.
    deployment_id : str
        The ID of the deployment to make predictions with.
    mimetype : str
        Mimetype describing data being sent.
        If mimetype starts with 'text/' or equal to 'application/json',
        data will be decoded with provided or default(UTF-8) charset
        and passed into the 'score_unstructured' hook implemented in custom.py provided with the model.

        In case of other mimetype values data is treated as binary and passed without decoding.
    charset : str
        Charset should match the contents of the file, if file is text.

    Returns
    -------
    data : bytes
        Arbitrary data returned by unstructured model.


    Raises
    ------
    DataRobotPredictionError if there are issues getting predictions from DataRobot
    """
    # Set HTTP headers. The charset should match the contents of the file.
    headers = {
        'Content-Type': '{};charset={}'.format(mimetype, charset),
        'Authorization': 'Bearer {}'.format(API_KEY),
        'DataRobot-Key': DATAROBOT_KEY,
    }

    url = API_URL.format(deployment_id=deployment_id)

    # Make API request for predictions
    predictions_response = requests.post(
        url,
        data=data,
        headers=headers,
    )
    _raise_dataroboterror_for_status(predictions_response)
    # Return raw response content
    return predictions_response.content


def _raise_dataroboterror_for_status(response):
    """Raise DataRobotPredictionError if the request fails along with the response returned"""
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        err_msg = '{code} Error: {msg}'.format(
            code=response.status_code, msg=response.text)
        raise DataRobotPredictionError(err_msg)


def datarobot_predict_file(filename, deployment_id, mimetype='text/csv', charset='utf-8'):
    """
    Return an exit code on script completion or error. Codes > 0 are errors to the shell.
    Also useful as a usage demonstration of
    `make_datarobot_deployment_unstructured_predictions(data, deployment_id, mimetype, charset)`
    """
    data = open(filename, 'rb').read()
    data_size = sys.getsizeof(data)
    if data_size >= MAX_PREDICTION_FILE_SIZE_BYTES:
        print((
                  'Input file is too large: {} bytes. '
                  'Max allowed size is: {} bytes.'
              ).format(data_size, MAX_PREDICTION_FILE_SIZE_BYTES))
        return 1
    try:
        predictions = make_datarobot_deployment_unstructured_predictions(data, deployment_id, mimetype, charset)
        return predictions
    except DataRobotPredictionError as exc:
        pprint(exc)
        return None

def datarobot_predict(input_dict, deployment_id, mimetype='application/json', charset='utf-8'):
    """
    Return an exit code on script completion or error. Codes > 0 are errors to the shell.
    Also useful as a usage demonstration of
    `make_datarobot_deployment_unstructured_predictions(data, deployment_id, mimetype, charset)`
    """
    data = json.dumps(input_dict).encode(charset)
    data_size = sys.getsizeof(data)
    if data_size >= MAX_PREDICTION_FILE_SIZE_BYTES:
        print((
                  'Input file is too large: {} bytes. '
                  'Max allowed size is: {} bytes.'
              ).format(data_size, MAX_PREDICTION_FILE_SIZE_BYTES))
        return 1
    try:
        predictions = make_datarobot_deployment_unstructured_predictions(data, deployment_id, mimetype, charset)
        return json.loads(predictions)['answer']
    except DataRobotPredictionError as exc:
        pprint(exc)
        return None

## Create deployment

In [ ]:
deployment = deploy_to_datarobot(BASE_PATH, 
                                 "Keras", 
                                 "blog-bert-tf-questionAnswering", 
                                 "Pretrained BERT model, fine-tuned on SQUAD for question answering")

Creating custom model version on ExecutionEnvironment('[DataRobot] Python 3.9 Keras Drop-In')...
Created CustomModelVersion('v1.0').
Building the execution environment with dependency packages...
Environment build completed with success.
Creating model deployment...
Deployment(blog-bert-tf-questionAnswering) is ready!


# Use in production

In [ ]:
test_input = {"abstract": "Healthcare tasks (e.g., patient care via disease treatment) and biomedical research (e.g., scientific discovery of new therapies) require expert knowledge that is limited and expensive. Foundation models present clear opportunities in these domains due to the abundance of data across many modalities (e.g., images, text, molecules) to train foundation models, as well as the value of improved sample efficiency in adaptation due to the cost of expert time and knowledge. Further, foundation models may allow for improved interface design (§2.5: interaction) for both healthcare providers and patients to interact with AI systems, and their generative capabilities suggest potential for open-ended research problems like drug discovery. Simultaneously, they come with clear risks (e.g., exacerbating historical biases in medical datasets and trials). To responsibly unlock this potential requires engaging deeply with the sociotechnical matters of data sources and privacy as well as model interpretability and explainability, alongside effective regulation of the use of foundation models for both healthcare and biomedicine.", "question": "Where can we use foundation models?"}
pprint(test_input)

{'abstract': 'Healthcare tasks (e.g., patient care via disease treatment) and '
             'biomedical research (e.g., scientific discovery of new '
             'therapies) require expert knowledge that is limited and '
             'expensive. Foundation models present clear opportunities in '
             'these domains due to the abundance of data across many '
             'modalities (e.g., images, text, molecules) to train foundation '
             'models, as well as the value of improved sample efficiency in '
             'adaptation due to the cost of expert time and knowledge. '
             'Further, foundation models may allow for improved interface '
             'design (§2.5: interaction) for both healthcare providers and '
             'patients to interact with AI systems, and their generative '
             'capabilities suggest potential for open-ended research problems '
             'like drug discovery. Simultaneously, they come with clear risks '
            

In [ ]:
datarobot_predict(test_input, deployment.id)

'both healthcare and biomedicine'